In [1]:
import sys
sys.path.insert(1, '../')


In [2]:
import importlib
import medical_explainer
importlib.reload(medical_explainer)


<module 'medical_explainer' from '/home/llja/repos/Uni_ACS/mimiciii_sepsis/../medical_explainer.py'>

In [3]:
import pandas as pd
from sklearn.pipeline import make_pipeline

from sklearn.model_selection import train_test_split

seed = 7

data = pd.read_csv('mimiciii_sepsis.csv').drop(['Unnamed: 0', 'SUBJECT_ID', 'HADM_ID'], axis=1)
data['Gender'] = data['Gender'].apply(lambda x: 1 if x == 'M' else 0)

X = data.drop(['28 Day Death', 'In Hospital Death'], axis=1)
y = data['28 Day Death']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=seed)

In [4]:
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imp = IterativeImputer(max_iter=10, random_state=0)
imp.fit(X_train)

X_train = pd.DataFrame(imp.transform(X_train), columns=X.columns)
X_test = pd.DataFrame(imp.transform(X_test), columns=X.columns)

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

# clf = MLPClassifier(random_state=seed, max_iter=300).fit(X_train_scaled, y_train)
# clf = MLPClassifier(random_state=seed, max_iter=1000, activation='logistic', solver='sgd').fit(X_train_scaled, y_train)

estimators = [('Scaler', StandardScaler()), ('Neural_Network', MLPClassifier(random_state=seed, max_iter=1000, activation='logistic', solver='sgd'))]
pipe = Pipeline(estimators)

clf = pipe.fit(X_train, y_train)

In [27]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import accuracy_score

roc_auc = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
print("ROC AUC: " + str(roc_auc))

average_precision = average_precision_score(y_test, clf.predict_proba(X_test)[:, 1])
print("Average Precision: " + str(average_precision))

accuracy = accuracy_score(y_test, clf.predict(X_test))
print("Accuracy: " + str(accuracy))

ROC AUC: 0.8095044991980281
Average Precision: 0.7386963052733777
Accuracy: 0.7556693489392831


In [28]:
explainer = medical_explainer.explainer(
    clf, X_train, y_train, X_test, y_test)
# explainer.plot_calibration_original()
# explainer.calibrate()
# explainer.plot_calibration_calibrated()


In [ ]:
explainer = medical_explainer.explainer(
    clf, X_train, y_train, X_test, y_test)

explainer.fit(10, shap_method='kernel', method='novel')

In [12]:
explainer.variables

array(['LOS', 'RDW', 'ICU_LOS', 'Age', 'inotropes',
       'Lactate Dehydrogenase (LD)', 'Haptoglobin', 'Phosphate', 'ICUs',
       'Albumin'], dtype=object)

In [13]:
explainer.score_array_list

[array([ 8., -5.]),
 array([-3.,  4.]),
 array([ 1., -2.,  4.]),
 array([-2.,  3.]),
 array([-3.,  2.]),
 array([-2.,  1.]),
 array([ 2., -1.]),
 array([-1.,  1.]),
 array([ 0., -1.,  4.]),
 array([ 1., -1.])]

In [14]:
explainer.breakpoints_list

[[-0.3222222222222222, 8.420296637338277, 294.66041666666666],
 [11.6, 15.442368481276056, 30.5],
 [0.008, 0.043143496038710594, 5.281013550406813, 153.928],
 [18.258726899383984, 73.08052113237441, 90.0],
 [0.0, 0.9401474154387959, 3.0],
 [-8.271937990619335, 207.60238455174488, 23111.0],
 [-151.44629140606094, 158.81343669848746, 688.0],
 [0.2, 3.4487048423198887, 17.3],
 [0.0, 0.26254122009811004, 1.121022950650504, 5.0],
 [0.0, 2.7469765455333497, 18.290282613036652]]

In [15]:
scores,probs,predictions = explainer.predict_calculator(X_test, 1)

In [16]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

roc_auc = roc_auc_score(y_test, probs)
print("ROC AUC: " + str(roc_auc))

average_precision = average_precision_score(y_test, probs)
print("Average Precision: " + str(average_precision))

# accuracy = accuracy_score(y_test, predictions)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: " + str(accuracy))

# print(confusion_matrix(y_test, predictions))
print(confusion_matrix(y_test, predictions))

ROC AUC: 0.7432410084003154
Average Precision: 0.6392679031444433
Accuracy: 0.695683979517191
[[772  72]
 [344 179]]


In [25]:
import numpy as np
import scipy.stats


def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h


In [30]:
from sklearn.utils import resample

auroc_array = []
auprc_array = []
accuracy_array = []

for i in range(50):
    X_test_bs = resample(X_test, n_samples=200, replace=False, stratify=y_test,
             random_state=i)
    y_test_bs = y_test[X_test_bs.index]

    scores,probs,predictions = explainer.predict_calculator(X_test_bs, 1)
    auroc = roc_auc_score(y_test_bs, probs)
    auprc = average_precision_score(y_test_bs, probs)
    accuracy = accuracy_score(y_test_bs, predictions)
    
    auroc_array.append(auroc)
    auprc_array.append(auprc)
    accuracy_array.append(accuracy)
    

print(mean_confidence_interval(auroc_array))
print(mean_confidence_interval(auprc_array))
print(mean_confidence_interval(accuracy_array))

(0.7469063456868335, 0.7376555577417688, 0.7561571336318981)
(0.6471622941122264, 0.6342367766283696, 0.6600878115960831)
(0.6975999999999999, 0.6902457637156658, 0.704954236284334)
